<a href="https://colab.research.google.com/github/Thiruvikraman07/AreaDetection/blob/main/ssd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="um9cTmvGD5l6LS1PMPBn")
project = rf.workspace("ptstry").project("boar-detection")
dataset = project.version(5).download("voc")

In [2]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import tensorflow as tf

In [3]:
!git clone --q https://github.com/tensorflow/models.git

In [4]:
%cd models/research

/content/models/research


In [5]:
!protoc object_detection/protos/*.proto --python_out=.

In [6]:
!cp object_detection/packages/tf2/setup.py .

In [7]:
!python -m pip install .

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

In [10]:
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      
      value = (root.find('filename').text,
                 int(root.find('size').find('width').text),
                 int(root.find('size').find('height').text),
                 member[0].text,
                 int(member.find("bndbox").find('xmin').text),
                 int(member.find("bndbox").find('ymin').text),
                 int(member.find("bndbox").find('xmax').text),
                 int(member.find("bndbox").find('ymax').text)
                )
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['/content/Boar-Detection-5/train', '/content/Boar-Detection-5/test']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join("/content/customTF2/label_map.pbtxt")
pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)
    print('Successfully created label_map.pbtxt ')

Successfully converted /content/Boar-Detection-5/train xml to csv.
Successfully converted /content/Boar-Detection-5/test xml to csv.
Successfully created label_map.pbtxt 


In [19]:
!python /content/customTF2/generate_tfrecord.py /content/Boar-Detection-5/train.csv  /content/customTF2/label_map.pbtxt /content/Boar-Detection-5/train /content/customTF2/train.record

2023-05-17 14:33:38.105296: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
groups: 100% 3968/3968 [00:03<00:00, 1116.08it/s]
Successfully created the TFRecords: /content/customTF2/train.record


In [20]:
!python /content/customTF2/generate_tfrecord.py /content/Boar-Detection-5/test.csv  /content/customTF2/label_map.pbtxt /content/Boar-Detection-5/test /content/customTF2/test.record

2023-05-17 14:34:45.281571: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
groups: 100% 190/190 [00:00<00:00, 1040.18it/s]
Successfully created the TFRecords: /content/customTF2/test.record


In [21]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

--2023-05-17 14:35:37--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.201.128, 2607:f8b0:4001:c5f::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.201.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  63.0MB/s    in 0.3s    

2023-05-17 14:35:38 (63.0 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]



In [22]:
!tar -xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index


In [25]:
!cd /usr/local/lib/python3.10/dist-packages/tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 85527, done.
remote: Counting objects: 100% (1126/1126), done.
remote: Compressing objects: 100% (470/470), done.
remote: Total 85527 (delta 690), reused 1053 (delta 650), pack-reused 84401
Receiving objects: 100% (85527/85527), 598.64 MiB | 24.77 MiB/s, done.
Resolving deltas: 100% (61192/61192), done.


In [26]:
!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz /content/Pre-trained

In [27]:
!cd /content/Pre-trained && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index


In [28]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [29]:
CONFIG_PATH = '/content/Pre-trained/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'

In [30]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [ ]:
config

In [32]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [34]:
pipeline_config.model.ssd.num_classes = 1
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = '/content/Pre-trained'+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= '/content/customTF2/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = ['/content/customTF2/train.record']
pipeline_config.eval_input_reader[0].label_map_path = '/content/customTF2/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = ['/content/customTF2/test.record']

In [35]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

In [53]:
!pip show tensorflow

Name: tensorflow
Version: 2.12.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, jax, keras, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine-rl, object-detection, tensorflow-text, tf-models-official


In [55]:
WORKSPACE_PATH = 'usr/local/lib/python3.10/dist-packages/tensorflow/workspace'
SCRIPTS_PATH = 'usr/local/lib/python3.10/dist-packages/tensorflow/scripts'
APIMODEL_PATH = 'usr/local/lib/python3.10/dist-packages/tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

In [56]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format('/content/models', MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))


python /content/models/research/object_detection/model_main_tf2.py --model_dir=usr/local/lib/python3.10/dist-packages/tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=usr/local/lib/python3.10/dist-packages/tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000


In [58]:
!cp object_detection/packages/tf2/setup.py .
!python3 -m pip install --user --use-feature=2020-resolver .


Usage:   
  /usr/bin/python3 -m pip install [options] <requirement specifier> [package-index-options] ...
  /usr/bin/python3 -m pip install [options] -r <requirements file> [package-index-options] ...
  /usr/bin/python3 -m pip install [options] [-e] <vcs project url> ...
  /usr/bin/python3 -m pip install [options] [-e] <local project path> ...
  /usr/bin/python3 -m pip install [options] <archive url/path> ...

option --use-feature: invalid choice: '2020-resolver' (choose from 'fast-deps', 'truststore', 'no-binary-enable-wheel-cache')


In [63]:
!pip show object_detection


Name: object-detection
Version: 0.0.3
Summary: A package build from Tensorflow's object detection API.
Home-page: https://github.com/brandonschabell/models/tree/master/research/object_detection
Author: Brandon Schabell
Author-email: brandonschabell@gmail.com
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: contextlib2, Cython, jupyter, lxml, matplotlib, pillow, tensorflow
Required-by: 


In [67]:
%cd content/models/research

[Errno 2] No such file or directory: 'content/models/research'
/content/models/research


In [68]:
!protoc object_detection/protos/*.proto --python_out=. 

In [70]:
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1696978 sha256=93cf2fb7510addf14f7ce99b814e0f52b1939605f93fefd1dfcc1aa32d619534
  Stored in directory: /tmp/pip-ephem-wheel-cache-mhif6ikm/wheels/53/dd/70/2de274d6c443c69d367bd6a5606f95e5a6df61aacf1435ec0d
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.0.3
    Uninstalling object-detection-0.0.3:
      Successfully uninstalled object-detection-0.0.3


In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py --model_dir=usr/local/lib/python3.10/dist-packages/tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=/content/Pre-trained/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config --num_train_steps=5000


2023-05-17 15:41:59.863639: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2023-05-17 15:42:03.228434: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0517 15:42:03.248091 139852702287680 mir

In [46]:
!pip uninstall numpy
!pip install numpy


Found existing installation: numpy 1.24.3
Uninstalling numpy-1.24.3:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.10
    /usr/local/lib/python3.10/dist-packages/numpy-1.24.3.dist-info/*
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libopenblas64_p-r0-15028c96.3.21.so
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.10/dist-packages/numpy/*
Proceed (Y/n)? Y
  Successfully uninstalled numpy-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.24.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<

In [ ]:
!pip uninstall object_detection
!pip uninstall numpy
!pip install numpy==1.19.5
!pip install object_detection

In [60]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

RuntimeError: ignored

ImportError: ignored

In [ ]:
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections